<a href="https://colab.research.google.com/github/jeet-yadav27/Deep_learning/blob/main/7_Develop_MLP_Time_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Keras libraries you've imported: Model, Input, Dense, and concatenate. Here's a breakdown of their roles in building neural networks with Keras:

keras.models.Model

This is the core class for defining a Keras model. There are two main ways to create a model:
Sequential API: Suitable for simple, linear stacks of layers (as used in earlier parts of your code).
Functional API: More flexible for complex models with multiple inputs, outputs, or shared layers (what you're likely moving towards now).
keras.layers.Input

Used in the Functional API to define an input layer for your model. It specifies the shape of the data your model will receive.
keras.layers.Dense

Represents a fully connected layer, where each neuron receives input from all neurons of the previous layer. Common arguments include:
units: The number of neurons in the layer.
activation: The activation function to apply (e.g., 'relu', 'sigmoid', 'softmax').
input_dim (for the first layer): The number of input features.
keras.layers.concatenate

Used to merge the outputs of multiple layers into a single tensor. This is essential in models with multiple input branches or when combining features from different parts of the network.
Example using the Functional API

Here's a basic example of how you might use these components together to create a model with two input branches:


from keras.models import Model
from keras.layers import Input, Dense, concatenate

### Define input layers
input1 = Input(shape=(10,))
input2 = Input(shape=(5,))

### Define branches
branch1 = Dense(32, activation='relu')(input1)
branch2 = Dense(16, activation='relu')(input2)

### Merge branches
merged = concatenate([branch1, branch2])

### Output layer
output = Dense(1, activation='sigmoid')(merged)

### Create the model
model = Model(inputs=[input1, input2], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy')

##How to Develop MLPs for Time Series
Forecasting

Tutorial Overview

1. Univariate MLP Models
2. Multivariate MLP Models
3. Multi-step MLP Models
4. Multivariate Multi-step MLP Models

### 1. Univariate MLP Models

1. Data Preparation
2. MLP Model

Step 1- Data Preparation

In [ ]:
# Split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # Find the end of this pattern
        end_ix = i + n_steps

        # Check if we are beyond the sequence
        if end_ix > len(sequence) - 1:
            break

        # Gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)

    return array(X), array(y)

# Define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

# Choose a number of time steps
n_steps = 3

# Split into samples
X, y = split_sequence(raw_seq, n_steps)

# Summarize the data
for i in range(len(X)):
    print( X[i],  y[i])

[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90


2. MLP Model

In [ ]:
# define
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_steps))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=2000, verbose=0)
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps))
yhat = model.predict(x_input, verbose=0)
print(yhat)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[[101.12477]]


In [ ]:
print(yhat) # Prediction

[[101.12477]]


### 2. Multivariate MLP Models

1. Multiple Input Series.
2. Multiple Parallel Series.

#### Multiple Input Series

In [ ]:
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
from numpy import array
from numpy import hstack

# Split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # Find the end of this pattern
        end_ix = i + n_steps

        # Check if we are beyond the dataset
        if end_ix > len(sequences):
            break

        # Gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)

    return array(X), array(y)

# Define input sequences
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

# Convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

# Horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))

# Choose a number of time steps
n_steps = 3

# Convert into input/output
X, y = split_sequences(dataset, n_steps)
print(X.shape, y.shape)

# Summarize the data
for i in range(len(X)):
    print(X[i], y[i])


(7, 3, 2) (7,)
[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


MLP Model
Before we can
t an MLP on this data, we must
fatten the shape of the input samples.

In [ ]:
# flatten input
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], n_input))

In [ ]:
X

array([[10, 15, 20, 25, 30, 35],
       [20, 25, 30, 35, 40, 45],
       [30, 35, 40, 45, 50, 55],
       [40, 45, 50, 55, 60, 65],
       [50, 55, 60, 65, 70, 75],
       [60, 65, 70, 75, 80, 85],
       [70, 75, 80, 85, 90, 95]])

In [ ]:
#deffine the Model

In [ ]:
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_input))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# fit model
model.fit(X, y, epochs=2000, verbose=0)


In [ ]:
# demonstrate prediction
x_input = array([[80, 85], [90, 95], [100, 105]])
x_input = x_input.reshape((1, n_input))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[205.61015]]


Multi-Headed MLP MODEL

A multi-headed MLP (Multilayer Perceptron) model is a type of neural network architecture that extends the traditional MLP by incorporating multiple output heads or branches. Each head is responsible for predicting a specific task or output, allowing the model to learn multiple related tasks simultaneously.

Architecture:

1. Input Layer: Receives input data.
2. Shared Encoder: A series of hidden layers that extract features from the input data.
3. Multiple Output Heads: Each head consists of one or more hidden layers and an output layer, specific to each task.

Example:

Suppose we want to predict:

1. House prices (regression task)
2. House types (classification task: apartment, house, condo)
3. House locations (classification task: urban, suburban, rural)

Multi-Headed MLP Model:

Input Layer (features: number of rooms, square footage, etc.)

Shared Encoder:

- Hidden Layer 1 (64 units, ReLU activation)
- Hidden Layer 2 (32 units, ReLU activation)

Output Heads:

Head 1: House Prices (Regression)

- Hidden Layer 3 (16 units, ReLU activation)
- Output Layer (1 unit, linear activation)

Head 2: House Types (Classification)

- Hidden Layer 4 (16 units, ReLU activation)
- Output Layer (3 units, softmax activation)

Head 3: House Locations (Classification)

- Hidden Layer 5 (16 units, ReLU activation)
- Output Layer (3 units, softmax activation)

Loss Functions:

1. Mean Squared Error (MSE) for house prices
2. Cross-Entropy Loss for house types and locations

Training:

The model is trained on a dataset containing input features and corresponding outputs for each task. The loss functions are combined using weighted sums or other techniques to balance the importance of each task.

Benefits:

1. Improved performance on individual tasks due to shared feature learning.
2. Reduced overfitting by leveraging relationships between tasks.
3. Efficient use of parameters, as shared encoder is used across tasks.

This model requires input to be provided as a list of two elements, where each element in
the list contains data for one of the submodels. In order to achieve this, we can split the 3D
input data into two separate arrays of input data: that is from one array with the shape [7, 3,
2] to two 2D arrays with the shape [7, 3].

In [ ]:
from numpy import array
from numpy import hstack
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
#from keras.layers.merge import
from keras.layers import concatenate # Updated import statement

# Split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # Find the end of this pattern
        end_ix = i + n_steps

        # Check if we are beyond the dataset
        if end_ix > len(sequences):
            break

        # Gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)

    return array(X), array(y)

# Define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

# Convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

# Horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))

# Choose a number of time steps
n_steps = 3

# Convert into input/output
X, y = split_sequences(dataset, n_steps)

# Separate input data
X1 = X[:, :, 0]
X2 = X[:, :, 1]

# First input model
visible1 = Input(shape=(n_steps,))
dense1 = Dense(100, activation='relu')(visible1)

# Second input model
visible2 = Input(shape=(n_steps,))
dense2 = Dense(100, activation='relu')(visible2)

# Merge input models
merge = concatenate([dense1, dense2])
output = Dense(1)(merge)

# Define model
model = Model(inputs=[visible1, visible2], outputs=output)

# Compile model
model.compile(optimizer='adam', loss='mse')

# Fit model
model.fit([X1, X2], y, epochs=2000, verbose=0)

# Demonstrate prediction
x_input = array([[80, 85], [90, 95], [100, 105]])
x1 = x_input[:, 0].reshape((1, n_steps))
x2 = x_input[:, 1].reshape((1, n_steps))
yhat = model.predict([x1, x2], verbose=0)

# Print prediction
print(yhat)

[[205.87367]]


### 2. Multiple Parallel Series

In [ ]:
from numpy import array
from numpy import hstack

# Split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # Find the end of this pattern
        end_ix = i + n_steps

        # Check if we are beyond the dataset
        if end_ix > len(sequences) - 1:
            break

        # Gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)

    return array(X), array(y)

# Define input sequences
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i] + in_seq2[i] for i in range(len(in_seq1))])

# Convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

# Horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))

# Choose a number of time steps
n_steps = 3

# Convert into input/output
X, y = split_sequences(dataset, n_steps)
print("X Shape:", X.shape)
print("y Shape:", y.shape)

# Summarize the data
for i in range(len(X)):
    print(X[i], y[i])


X Shape: (6, 3, 3)
y Shape: (6, 3)
[[10 15 25]
 [20 25 45]
 [30 35 65]] [40 45 85]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [ 50  55 105]
[[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] [ 60  65 125]
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] [ 70  75 145]
[[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] [ 80  85 165]
[[ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]] [ 90  95 185]


 Vector- Output MLP

We are now ready to
t an MLP model on this data

The model output will be a vector, with one element for each of the three di
erent time
series

In [ ]:
# flatten input
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], n_input))
n_output = y.shape[1]

In [ ]:
n_output

3

In [ ]:

# define model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_input))
model.add(Dense(n_output))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=2000, verbose=0)
# demonstrate prediction
x_input = array([[70,75,145], [80,85,165], [90,95,185]])
x_input = x_input.reshape((1, n_input))
yhat = model.predict(x_input, verbose=0)
print(yhat)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[[ 99.999916 104.999916 205.00098 ]]


Multi-Output MLP Model

In [4]:
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input
from keras.models import Model

# Split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # Find the end of this pattern
        end_ix = i + n_steps

        # Check if we are beyond the dataset
        if end_ix > len(sequences) - 1:
            break

        # Gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)

    return array(X), array(y)

# Define input sequences
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i] + in_seq2[i] for i in range(len(in_seq1))])

# Convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

# Horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))

# Choose a number of time steps
n_steps = 3

# Convert into input/output
X, y = split_sequences(dataset, n_steps)

# Flatten input
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], n_input))

# Define number of outputs
n_output = y.shape[1]

# separate output
y1 = y[:, 0].reshape((y.shape[0], 1))
y2 = y[:, 1].reshape((y.shape[0], 1))
y3 = y[:, 2].reshape((y.shape[0], 1))
# define model
visible = Input(shape=(n_input,))

dense = Dense(100, activation='relu')(visible)
# define output 1
output1 = Dense(1)(dense)
# define output 2
output2 = Dense(1)(dense)
# define output 2
output3 = Dense(1)(dense)
# tie together
model = Model(inputs=visible, outputs=[output1, output2, output3])
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, [y1,y2,y3], epochs=2000, verbose=0)
# demonstrate prediction
x_input = array([[70,75,145], [80,85,165], [90,95,185]])
x_input = x_input.reshape((1, n_input))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[array([[100.90562]], dtype=float32), array([[106.528114]], dtype=float32), array([[207.68948]], dtype=float32)]


### 7.4 Multi-step MLP Models

Data Preparation

In this We been to Set the Three Input Variable to make the prediction of Next Two Variable

In [5]:
# Univariate multi-step vector-output MLP example

from numpy import array
from keras.models import Sequential
from keras.layers import Dense

# Split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        # Find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out

        # Check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break

        # Gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)

    return array(X), array(y)

# Define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

# Choose a number of time steps
n_steps_in, n_steps_out = 3, 2

# Split into samples
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)

# Define model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_steps_in))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')

# Fit model
model.fit(X, y, epochs=2000, verbose=0)

# Demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps_in))
yhat = model.predict(x_input, verbose=0)

print(yhat)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[[103.99346  116.641624]]


### 7.5 Multivariate Multi-step MLP Models

1. Multiple Input Multi-step Output.
2. Multiple Parallel Input and Multi-step Output.

7.5.1 Multiple Input Multi-step Output

In [9]:
# multivariate multi-step mlp example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out - 1

        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break

        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)

    return array(X), array(y)

# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# convert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
# flatten input
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], n_input))
# define model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_input))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=2000, verbose=0)
# demonstrate prediction
x_input = array([[70, 75], [80, 85], [90, 95]])
x_input = x_input.reshape((1, n_input))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[186.86073 208.58675]]


7.5.2 Multiple Parallel Input and Multi-step Output

In [10]:
# Multivariate multi-step MLP example

from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense

# Split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out

        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break

        # gather input and output parts of the sequence
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)

    return array(X), array(y)

# Define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

# Convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

# Horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))

# Choose a number of time steps
n_steps_in, n_steps_out = 3, 2

# Convert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)

# Flatten input
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], n_input))

# Flatten output
n_output = y.shape[1] * y.shape[2]
y = y.reshape((y.shape[0], n_output))

# Define model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_input))
model.add(Dense(n_output))
model.compile(optimizer='adam', loss='mse')

# Fit model
model.fit(X, y, epochs=2000, verbose=0)

# Demonstrate prediction
x_input = array([[60, 65, 125], [70, 75, 145], [80, 85, 165]])
x_input = x_input.reshape((1, n_input))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[ 91.11308  96.57112 186.98349 100.90645 107.41045 208.92159]]


Here's a theoretical explanation of developing MLP models for univariate, multivariate, and multi-step time series forecasting, highlighting the differences:

Univariate Time Series Forecasting

- Forecasting a single variable based on its past values.
- Goal: Predict future values of the same variable.

MLP Architecture:

1. Input Layer: Lagged values of the single variable (e.g., t-1, t-2, ..., t-n).
2. Hidden Layers: Nonlinear transformations (e.g., ReLU, Tanh).
3. Output Layer: Single forecasted value.

Key Considerations:

1. Lag selection: Choose relevant lagged values.
2. Window size: Determine the number of past values.
3. Model complexity: Balance between underfitting and overfitting.

Example:

Forecasting daily temperature using past temperature values.

Multivariate Time Series Forecasting

- Forecasting multiple variables based on their past values and relationships.
- Goal: Predict future values of multiple variables.

MLP Architecture:

1. Input Layer: Lagged values of multiple variables (e.g., temperature, humidity, wind).
2. Hidden Layers: Nonlinear transformations.
3. Output Layer: Multiple forecasted values.

Key Considerations:

1. Feature selection: Choose relevant variables.
2. Correlation analysis: Understand relationships between variables.
3. Model complexity: Balance between underfitting and overfitting.

Example:

Forecasting daily temperature, humidity, and wind using past values of these variables.

Multi-Step Time Series Forecasting

- Forecasting multiple future values of a variable(s) based on past values.
- Goal: Predict multiple future values.

MLP Architecture:

1. Input Layer: Lagged values of the variable(s).
2. Hidden Layers: Nonlinear transformations.
3. Output Layer: Multiple forecasted values (e.g., t+1, t+2, ..., t+n).

Key Considerations:

1. Horizon selection: Choose the number of future steps.
2. Sequence-to-sequence modeling: Use encoder-decoder architectures.
3. Error accumulation: Address compounding errors.

Example:

Forecasting daily temperature for the next 7 days using past temperature values.

Comparison of MLP Architectures:

|  | Univariate | Multivariate | Multi-Step |
| --- | --- | --- | --- |
| Input | Single variable | Multiple variables | Single/Multiple variables |
| Output | Single value | Multiple values | Multiple values |
| Complexity | Lower | Higher | Higher |
| Considerations | Lag selection, window size | Feature selection, correlation | Horizon selection, sequence-to-sequence |

In summary:

- Univariate MLPs focus on a single variable's past values.
- Multivariate MLPs consider multiple variables and their relationships.
- Multi-step MLPs predict multiple future values, addressing sequence-to-sequence challenges.

When developing MLP models, consider the specific characteristics of your time series data and adjust the architecture accordingly.